In [54]:
import numpy as np
import sys
import pandas as pd
sys.path.insert(0, "/Users/mallen2/OpenPATH_Data/e-mission-server") 

from pymongo import MongoClient
client = MongoClient()
db = client.Stage_database

import emission.storage.timeseries.aggregate_timeseries as estag 
agts = estag.AggregateTimeSeries()   

inferred_trip_query = db.Stage_analysis_timeseries.find({
    "$and": [ {"metadata.key":"analysis/confirmed_trip"},
            {"data.algorithm_chosen": {"$in": ["label_assist","sensing"]}}]
    }) 

inferred_trips_df = agts.to_data_df("analysis/confirmed_trip",inferred_trip_query)

LABEL_CATEGORIES = ['mode_confirm','purpose_confirm','replaced_mode']

accepted_labels = {} # dictionary by label type with a list of standard labels for each type. Used to group custom labels as "Other"
#error_rates = db.Error_Rates.find_one()
#label_assist_mode_count_variances = error_rates["label_assist_mode_count_variances"]

accepted_labels["mode_confirm"] = ["Walk", "Bike","e-bike","Scooter share", "Drove Alone", "Shared Ride", "Taxi/Uber/Lyft", "Bus", "Train", "Free Shuttle", "Air"]
intensity_df = pd.read_csv("/Users/mallen2/OpenPATH_Data/em-public-dashboard/viz_scripts/auxiliary_files/energy_intensity.csv")

In [55]:
def make_rel_errors(max_size):
    upper= np.random.rand(1)*max_size
    lower = -np.random.rand(1)*max_size
    return  [float(lower),float(upper)]

Error_Rates = db["Error_Rates"]
# mode count intervals, (purpose count intervals later), carbon intervals, energy intervals
# Maybe eventually provide the bootstrap distributions?
modes = ['Walk', 'Bike', 'e-bike', 'Scooter share', 'Drove Alone', 'Shared Ride', 'Taxi/Uber/Lyft', 'Bus', 'Train', 'Free Shuttle', 'Air', 'Other','Total']
mode_rel_errors = {}
carbon_rel_errors = {}
energy_rel_errors = {}
for m in modes:
    if m is not 'Total':
        mode_rel_errors[m] = make_rel_errors(0.15)
    carbon_rel_errors[m] = make_rel_errors(0.2)
    energy_rel_errors[m] = make_rel_errors(0.2)

error_dictionary = {}
error_dictionary["counts"] = {}
error_dictionary["counts"]["mode_confirm"] = mode_rel_errors
error_dictionary["carbon"] = carbon_rel_errors
error_dictionary["energy"] = energy_rel_errors
error_dictionary["distance"] = {"interval": make_rel_errors(0.05), "mean": 0.01, "variance":0.02}

# to reset: db.Error_Rates.drop()

In [52]:
# Run this once. If you want to change it, drop the collection and rerun the above cell.
Error_Rates.insert_one(error_dictionary)

### Compute Metrics with Error Bars
The following functions compute metrics and error bars using inferred trip labels. These can be added to the values calculated from trips that have user input.

In [1]:
import count_functions as counts

db_errors = db.Error_Rates.find_one()
rel_count_errors = db_errors["counts"]
all_inferred_counts = counts.get_inferred_counts(inferred_trips_df)
count_intervals = counts.get_count_intervals(all_inferred_counts,LABEL_CATEGORIES, rel_count_errors)

NameError: name 'db' is not defined

In [83]:
count_intervals

{'mode_confirm': {'Other': {'count': 128,
   'interval': [122.08031960917052, 132.29127208835874]},
  'Drove Alone': {'count': 34,
   'interval': [29.594610426589405, 36.39919660047995]},
  'Walk': {'count': 9, 'interval': [8.7926140384392, 9.801002572005041]},
  'Bike': {'count': 2, 'interval': [1.9719114119104852, 2.117864092900778]}}}

In [ ]:
errors = db.Error_Rates.find_one()
errors

In [115]:
import impact_metrics as im

carbon_rel_errors = db_errors["carbon"]
energy_rel_errors = db_errors["energy"]
distances_by_mode = im.get_distances_by_mode(inferred_trips_df)
carbon_by_mode = im.get_inferred_carbon_intervals(distances_by_mode, carbon_rel_errors,intensity_df)
energy_by_mode = im.get_inferred_energy_intervals(distances_by_mode, energy_rel_errors,intensity_df)

# are the distances in meters in the database?

In [114]:
energy_by_mode

{'Drove Alone': {'estimate': 2592087647.3392005,
  'interval': [2515503002.216092, 3103583055.347957]},
 'Walk': {'estimate': 0.0, 'interval': [0.0, 0.0]},
 'Bike': {'estimate': 0.0, 'interval': [0.0, 0.0]}}

In [112]:
distances_by_mode

{'Other': 576713.500697305,
 'Drove Alone': 501370.91824742756,
 'Walk': 95361.01732049005,
 'Bike': 14852.071173176655,
 'Total': 1188297.5074383991}

In [46]:
intensity_df #[intensity_df["mode"] == "Car, drove alone"]["CO2_factor"]

,mode,fuel,(kWH)/trip,EI(kWH/PMT),energy_intensity_factor,energy_intensity_units,CO2_factor,CO2_factor_units
0,"Car, drove alone",gasoline,0.0000,NaN,5170.000,BTU/PMT,157.2,lb_CO2/MMBTU
1,"Car, with others",gasoline,0.0000,NaN,2585.000,BTU/PMT,157.2,lb_CO2/MMBTU
2,Taxi/Uber/Lyft,gasoline,0.0000,NaN,7214.000,BTU/PMT,157.2,lb_CO2/MMBTU
3,Bus,diesel,0.0000,NaN,4560.000,BTU/PMT,161.3,lb_CO2/MMBTU
4,Free Shuttle,diesel,0.0000,NaN,4560.000,BTU/PMT,161.3,lb_CO2/MMBTU
5,Train,electric,0.0000,0.370,0.370,kWH/PMT,1166.0,lb_CO2/MWH
6,Scooter share,electric,0.0041,0.027,0.027,kWH/PMT,1166.0,lb_CO2/MWH
7,Pilot ebike,electric,0.0000,0.022,0.022,kWH/PMT,1166.0,lb_CO2/MWH
8,Bikeshare,human_powered,0.0900,0.000,0.000,kWH/PMT,1166.0,lb_CO2/MWH
9,Walk,human_powered,0.0000,NaN,0.000,NaN,0.0,0
